In [178]:
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.gaussian_process.kernels import RationalQuadratic, Exponentiation
from sklearn.gaussian_process.kernels import PairwiseKernel
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import Matern
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np 

df_org = pd.read_csv('dataset_20211108.csv')
df_org = df_org[df_org.columns.difference(['Location'])]
df = df_org.dropna()

df = df[:-2]

print(df)
#df = df[['A11', 'A12', 'A21', 'A22', 'D1', 'D2', 'vonMises']]
X = np.array(df[['입력하중', 'PIPE 길이', 'PIPE 직경', '용접길이', 'A11', 'A12', 'A21', 'A22', 'D1', 'D2']])
y = np.array(df[['vonMises']])
#kernel = DotProduct()
#kernel = RBF() + ConstantKernel(constant_value=1)
#kernel = Exponentiation(RationalQuadratic(), exponent=2)
#kernel = PairwiseKernel(metric='rbf')
#kernel = RationalQuadratic(length_scale=1.0, alpha=1.5)
kernel = 1.0 * Matern(length_scale=1.0, nu=1.77)
gpr = GaussianProcessRegressor(kernel=kernel,  random_state=0).fit(X, y)
print(f'평가지표(the coefficient of determination of the prediction, r^2, 1 is best): {gpr.score(X, y)}')

test = df_org[-1:]
test_true = test['vonMises']
test =  np.array(test[['입력하중', 'PIPE 길이', 'PIPE 직경', '용접길이', 'A11', 'A12', 'A21', 'A22', 'D1', 'D2']])
test_pred, test_std = gpr.predict(test, return_std=True)
print(test)
print(f'true value: {test_true.values[0]}, predicted value: {test_pred[0][0]}, std: {test_std}')




## test set만 따로 이렇게 만들어서 하시는게 더 편하실 것 같아요!

test = pd.read_csv('dataset_test_20211108.csv') # test = df_org[-2:] 
pred_list = []
std_list = []
for i in range(len(test)):
    print()
    
    test_x =  np.array(test.iloc[i][['입력하중', 'PIPE 길이', 'PIPE 직경', '용접길이', 'A11', 'A12', 'A21', 'A22', 'D1', 'D2']])
    print(test_x)
    test_true = test.iloc[i]['vonMises']

    test_pred, test_std = gpr.predict([test_x], return_std=True)
    test_pred = test_pred[0][0]
    test_std = test_std[0]
    print(f'true value: {test_true}, predicted value: {test_pred}, std: {test_std}')
    pred_list.append(test_pred)
    std_list.append(test_std)


test = test[['입력하중', 'PIPE 길이', 'PIPE 직경', '용접길이', 'A11', 'A12', 'A21', 'A22', 'D1', 'D2', 'vonMises']]
test['predict'] = pred_list
test['std'] = std_list
test.to_csv('20211108_test_result.csv', index=False)
test


    A11  A12  A21  A22  D1  D2  PIPE 길이  PIPE 직경  vonMises  용접길이  입력하중
0     0    0    0    0  40  68      300       45     262.9    25   100
1    40    0    0    0  40  68      300       45     288.6    25   100
2     0   40    0    0  40  68      300       45     267.6    25   100
3     0    0  -40    0  40  68      300       45     350.8    25   100
4     0    0    0  -40  40  68      300       45     233.5    25   100
5     0    0  -89    0  40  68      300       45     361.6    25   100
6     0    0  -89    0  20  50      300       45     242.8    25   100
7    40   40    0    0  40  68      300       45     281.4    25   100
8    40    0  -40    0  40  68      300       45     371.4    25   100
9    40    0    0  -40  40  68      300       45     248.5    25   100
10    0   40  -40    0  40  68      300       45     350.8    25   100
11    0   40    0  -40  40  68      300       45     212.2    25   100
12    0    0  -40  -40  40  68      300       45     293.0    25   100
13   4

/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  

,입력하중,PIPE 길이,PIPE 직경,용접길이,A11,A12,A21,A22,D1,D2,vonMises,predict,std
0,100,300,45,25,0,60,-40,0,40,68,NaN,336.068813,14.464345
1,100,300,45,25,0,-20,-40,0,40,68,NaN,329.446829,13.974504
2,100,300,45,25,60,40,0,0,40,68,NaN,285.752878,13.962593
3,100,300,45,25,60,0,-20,0,40,68,NaN,333.137999,10.827628
4,100,300,45,25,-10,0,-20,0,40,68,NaN,308.187973,8.246480
5,100,300,45,25,0,0,0,60,40,68,145.817,172.242510,18.879081
6,100,300,45,25,0,0,0,90,40,68,63.605,123.689282,54.166441



[100 300 45 25 0 60 -40 0 40 68]
true value: nan, predicted value: 336.06881335006756, std: 14.464345061742431

[100 300 45 25 0 -20 -40 0 40 68]
true value: nan, predicted value: 329.44682911865766, std: 13.974504199949648

[100 300 45 25 60 40 0 0 40 68]
true value: nan, predicted value: 285.7528780261646, std: 13.962593279868182

[100 300 45 25 60 0 -20 0 40 68]
true value: nan, predicted value: 333.13799897499666, std: 10.82762849158529

[100 300 45 25 -10 0 -20 0 40 68]
true value: nan, predicted value: 308.18797275303086, std: 8.246479578501654

[100 300 45 25 0 0 0 60 40 68]
true value: 145.817, predicted value: 172.24251049121722, std: 18.879081026204997

[100 300 45 25 0 0 0 90 40 68]
true value: 63.605, predicted value: 123.68928243902474, std: 54.166441149187406


/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  

,입력하중,PIPE 길이,PIPE 직경,용접길이,A11,A12,A21,A22,D1,D2,vonMises,predict,std
0,100,300,45,25,0,60,-40,0,40,68,NaN,336.068813,14.464345
1,100,300,45,25,0,-20,-40,0,40,68,NaN,329.446829,13.974504
2,100,300,45,25,60,40,0,0,40,68,NaN,285.752878,13.962593
3,100,300,45,25,60,0,-20,0,40,68,NaN,333.137999,10.827628
4,100,300,45,25,-10,0,-20,0,40,68,NaN,308.187973,8.246480
5,100,300,45,25,0,0,0,60,40,68,145.817,172.242510,18.879081
6,100,300,45,25,0,0,0,90,40,68,63.605,123.689282,54.166441


(array([[252.48446312]]), array([1.01904932]))

In [7]:
df_org = pd.read_csv('dataset_20211108.csv')
df = df_org.dropna()
df

,입력하중,PIPE 길이,PIPE 직경,용접길이,A11,A12,A21,A22,D1,D2,Location,vonMises
0,100,300,45,25,0,0,0,0,40,68,PIPE,262.900
1,100,300,45,25,40,0,0,0,40,68,PIPE,288.600
2,100,300,45,25,0,40,0,0,40,68,PIPE,267.600
3,100,300,45,25,0,0,-40,0,40,68,PIPE,350.800
4,100,300,45,25,0,0,0,-40,40,68,PIPE,233.500
5,100,300,45,25,0,0,-89,0,40,68,PIPE,361.600
6,100,300,45,25,0,0,-89,0,20,50,PIPE,242.800
7,100,300,45,25,40,40,0,0,40,68,PIPE,281.400
8,100,300,45,25,40,0,-40,0,40,68,PIPE,371.400
9,100,300,45,25,40,0,0,-40,40,68,PIPE,248.500


In [263]:
#%%
# example of bayesian optimization for a 1d function from scratch
from math import sin
from math import pi
from numpy import arange
from numpy import vstack
from numpy import argmax
from numpy import asarray
from numpy.random import normal
from numpy.random import random
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from warnings import catch_warnings
from warnings import simplefilter
from matplotlib import pyplot
import random
#%%
# objective function
def objective(x, noise=0.1):
    noise = normal(loc=0, scale=noise)
	#y = (x**2 * sin(5 * pi * x)**6.0) 
    y = gpr.predict(np.array([x]), return_std=True)[0][0]+noise
	#y = -(x**4-x**2)+noise
	#print(f"'x': {x} | 'y': {y}")
    return y

 
# surrogate or approximation for the objective function
def surrogate(model, X):
	# catch any warning generated when making a prediction
	with catch_warnings():
		# ignore generated warnings
		simplefilter("ignore")
		return model.predict([X], return_std=True)
 
# probability of improvement acquisition function
def acquisition(X, Xsamples, model):
	# calculate the best surrogate score found so far
	best_list = []
	for X_elm in X:
		yhat, _ = surrogate(model, X_elm)
		best_list.append(yhat)
	best = max(best_list)
	# calculate mean and stdev via surrogate function
	# mu, std = surrogate(model, Xsamples)
	# mu = mu[:, 0]

	mu = []
	for sample in Xsamples:
		m, std = surrogate(model, sample)
		#print(m[0])
		mu.append(m[0])
	
	# calculate the probability of improvement
	probs = norm.cdf((mu - best) / (std+1E-9))
	return probs
 
# optimize the acquisition function
def opt_acquisition(X, y, model):
	# random search, generate random samples
	# Xsamples = random(100, 10)
	# Xsamples = Xsamples.reshape(len(Xsamples), 1)
	Xsamples = []
	for i in range(10):
		samples = []
		samples.append(100)
		samples.append(300)
		samples.append(45)
		samples.append(25)
		samples.append(random.randint(0, 50))
		samples.append(random.randint(0, 50))
		samples.append(-random.randint(0, 50))
		samples.append(-random.randint(0, 50))
		samples.append(random.randint(20, 50))
		samples.append(random.randint(50, 80))
		Xsamples.append(samples)
	Xsamples
	# calculate the acquisition function for each sample
	scores = acquisition(X, Xsamples, model)
	# locate the index of the largest scores
	ix = argmax(scores)

	return Xsamples[ix]
 
# plot real observations vs surrogate function
def plot(X, y, model):
	# scatter plot of inputs and real objective function
	pyplot.scatter(X, y)
	# line plot of surrogate function across domain
	Xsamples = asarray(arange(0, 1, 0.001))
	Xsamples = Xsamples.reshape(len(Xsamples), 1)
	ysamples, _ = surrogate(model, Xsamples)
	pyplot.plot(Xsamples, ysamples)
	# show the plot
	pyplot.show()
 


In [296]:
df = pd.read_csv('dataset_sejong.csv')
df = df.dropna()
df = df.iloc[[0, 1, 2], :]
df

,Unnamed: 0,입력하중,PIPE 길이,PIPE 직경,용접길이,A11,A12,A21,A22,D1,D2,Location,vonMises
0,CASE0,100,300,45,25,0,0,0,0,40,68,PIPE,262.887
1,CASE1,100,300,45,25,40,0,0,0,40,68,PIPE,288.555
2,CASE2,100,300,45,25,0,40,0,0,40,68,PIPE,267.592


In [301]:
# sample the domain sparsely with noise
#X = random(100)*100
#y = asarray([objective(x) for x in X])
df = pd.read_csv('dataset_sejong.csv')
df = df.dropna()
df = df.iloc[[0], :]
X = np.array(df[['입력하중', 'PIPE 길이', 'PIPE 직경', '용접길이', 'A11', 'A12', 'A21', 'A22', 'D1', 'D2']])
y = asarray([objective(x)[0] for x in X])
y = y.reshape((len(y), 1))
# reshape into rows and cols
#X = X.reshape(len(X), 1)
#y = y.reshape(len(y), 1)
# define the model
model = GaussianProcessRegressor()
# fit the model
model.fit(X, y)
# plot before hand
#plot(X, y, model)
# perform the optimization process
for i in range(300):
	# select the next point to sample
	x = opt_acquisition(X, y, model)
	# sample the point
	actual = objective(x)[0]
	# summarize the finding
	est, _ = surrogate(model, x)
	print(f'>x={x}, f()={est[0]}, actual={actual}')
	# add the data to the dataset
	X = vstack((X, np.array([x])))
	#print(y)
	y = vstack((y, np.array(actual))) # vstack((y, actual))
	#print(X)
	#print(y)
	# update the model
	model.fit(X, y)
 
# plot all samples and the final surrogate function
# plot(X, y, model)
# best result
ix = argmax(y)

print(f'Best Result: x={X[ix]}, y={y[ix]}')
# %%


>x=[100, 300, 45, 25, 42, 24, -7, -31, 50, 59], f()=[0.], actual=276.20823808344153
>x=[100, 300, 45, 25, 21, 30, -27, -25, 20, 68], f()=[0.], actual=276.066811728192
>x=[100, 300, 45, 25, 24, 30, -7, -33, 47, 79], f()=[3.85850771e-166], actual=276.05538253119994
>x=[100, 300, 45, 25, 47, 13, -26, -12, 43, 57], f()=[2.80949143e-198], actual=276.0982557273776
>x=[100, 300, 45, 25, 43, 49, -24, -25, 40, 79], f()=[2.33075911e-242], actual=276.2488006098318
>x=[100, 300, 45, 25, 11, 22, -30, -33, 42, 80], f()=[2.13289929e-169], actual=276.0697316386167
>x=[100, 300, 45, 25, 23, 21, -36, -49, 42, 57], f()=[4.7509891e-208], actual=276.1651267946346
>x=[100, 300, 45, 25, 8, 36, -38, -23, 29, 56], f()=[8.40031306e-119], actual=276.0606456067053
>x=[100, 300, 45, 25, 42, 13, -36, -6, 49, 53], f()=[1.54419215e-44], actual=275.8807904230986
>x=[100, 300, 45, 25, 19, 24, -39, -26, 38, 72], f()=[1.18600306e-58], actual=276.01975360554866
>x=[100, 300, 45, 25, 12, 1, -6, -36, 50, 71], f()=[3.3708768

In [302]:
df
ix

172

In [303]:
x=[100, 300, 45, 25, 7, 42, -1, -34, 28, 79]
objective(x)

array([276.00160235])

In [280]:
y.reshape(len(y),1).tolist()

[[262.84006067269553],
 [288.6451698060056],
 [267.6715142340702],
 [350.80243116938914],
 [233.45198571871683],
 [361.547629472424],
 [242.91893523360383],
 [281.5767491282219],
 [371.4319912288067],
 [248.5303482878013],
 [350.966092081353],
 [212.32255671309883],
 [116.57646224096459],
 [275.9773575146679],
 [275.9044540728736],
 [276.1163324848295],
 [276.1544217783036],
 [275.9017411448175],
 [276.16860162885155],
 [275.9533746935618],
 [276.0826904799455],
 [276.17668548941816],
 [275.9182011863458],
 [276.05404015672667],
 [276.0686120449681],
 [275.9233441445458],
 [276.13376696547743],
 [275.9936149913735],
 [276.13501194504425],
 [275.94620324479433],
 [276.23439763958925],
 [276.0565082239404],
 [275.8705965950308],
 [276.07483112626227],
 [275.97189610475647],
 [276.14614652160395],
 [276.1454306856015],
 [275.99331957093966],
 [276.0064481896523],
 [275.8163200387682],
 [276.01618789659665],
 [276.0610610898929],
 [276.0457561205462],
 [276.0601280349612],
 [276.2238105990